In [1]:
# Set this variable to true if ffmpeg is installed on your computer
ffmpeg_installed = true

true

In [2]:
using Printf
using Random
using LinearAlgebra
using PyPlot
using PyCall
@pyimport matplotlib.animation as anim

rng = MersenneTwister(18);

ArgumentError: ArgumentError: Package PyPlot not found in current path:
- Run `import Pkg; Pkg.add("PyPlot")` to install the PyPlot package.


In [3]:
# Size of matrix
n = 16
Q0 = rand(rng, n, n)
Λ = diagm(0 => Float64[1.4^(-i) for i=0:n-1])

F = qr(copy(Q0)); Q = F.Q
Λ[n,n] = 1e-1 * Λ[n,n]

A = Q * Λ / Q

UndefVarError: UndefVarError: rng not defined

In [4]:
Tk = copy(A)
for k=1:10
    F = qr(Tk); Uk = F.Q; Rk = F.R;
    Tk = Rk * Uk
end

# Print approx. and exact evalues
n_prt = 3
Tkdiag = diag(Tk)
exact = diag(Λ)
println("Tk    ",Tkdiag[1:n_prt])
println("Exact ",exact[1:n_prt])

UndefVarError: UndefVarError: A not defined

In [5]:
function record_snapshot(A, ims)
    """Saves figure snapshot for the animation"""
    im = imshow(copy(A), interpolation="nearest",
                extent=(0.5,size(A,1)+0.5,size(A,2)+0.5,0.5))
    clim(-8,1)
    push!(ims, PyCall.PyObject[im])
end

function create_animation(A, filename)
    fig = figure()
    ims = []

    Tk = copy(A)
    record_snapshot(log.(abs.(A))/log(10), ims)
    for k=1:70
        Uk, Rk = qr(Tk)
        Tk = Rk * Uk
        record_snapshot(log.(abs.(Tk))/log(10), ims) # This is used to make the movie
    end
    colorbar()
    ani = anim.ArtistAnimation(fig, ims, interval=100, blit=true)
    ani[:save](filename, extra_args=["-vcodec", "libx264", "-pix_fmt", "yuv420p"])
end

# Create animation for QR iteration
if ffmpeg_installed && ( !isfile("QR_iteration.mp4") || !isfile("QR_iteration_unsym.mp4") )

    F = qr(copy(Q0)); Q = F.Q
    A = Q * Λ / Q
    create_animation(A, "QR_iteration.mp4")
    
    A = Q0 * Λ / Q0
    create_animation(A, "QR_iteration_unsym.mp4")    
    
end

In [6]:
using Base64
function html_video(filename)
    open(filename) do f
        base64_video = base64encode(f)
        """<video controls src="data:video/x-m4v;base64,$base64_video">"""
    end
end

html_video (generic function with 1 method)

In [7]:
# Symmetric case
if isfile("QR_iteration.mp4") # Checks whether movie file exists
    display("text/html", html_video("QR_iteration.mp4"))
end

In [8]:
# Unsymmetric case
if isfile("QR_iteration_unsym.mp4") # Checks whether movie file exists
    display("text/html", html_video("QR_iteration_unsym.mp4"))
end